In [15]:
# =========================================
# 0. 라이브러리 & 기본 설정
# =========================================
import torch
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import CrossEncoder, InputExample
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


# =========================================
# 1. 학습 데이터 불러오기
# =========================================
dataset = load_dataset("HJUNN/Art_Therapy_caption_train_dataset")["train"]
train_df = dataset.to_pandas()

positive_pairs = list(zip(train_df["query"], train_df["doc"]))
print("Positive pairs:", len(positive_pairs))


# =========================================
# 2. 하드 네거티브 생성
# =========================================
from sentence_transformers import SentenceTransformer, util

embed = SentenceTransformer("HJUNN/bge-m3b-Art-Therapy-embedding-fine-tuning")

all_docs = list(dict.fromkeys(train_df["doc"].tolist()))
doc_emb = embed.encode(all_docs, convert_to_tensor=True, normalize_embeddings=True)

from collections import defaultdict
negatives = defaultdict(list)

for q, pos_doc in tqdm(positive_pairs):
    q_emb = embed.encode(q, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(q_emb, doc_emb)[0]
    top_k = torch.topk(scores, 10)

    for idx in top_k.indices:
        cand = all_docs[idx]
        if cand != pos_doc:
            negatives[q].append(cand)


# =========================================
# 3. Triples 생성 (query, pos, neg)
# =========================================
triples = []
for q, pos in positive_pairs:
    negs = negatives[q]
    if len(negs) == 0:
        continue
    selected = random.sample(negs, min(2, len(negs)))

    for neg in selected:
        triples.append((q, pos, neg))

print("Triples:", len(triples))


# =========================================
# 4. Dataset & DataLoader
# =========================================
class TripleDataset(Dataset):
    def __init__(self, triples):
        self.triples = triples

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        q, pos, neg = self.triples[idx]
        return q, pos, neg


dataset = TripleDataset(triples)
loader = DataLoader(dataset, batch_size=4, shuffle=True)


# =========================================
# 5. CrossEncoder 로드
# =========================================
model_name = "BAAI/bge-reranker-base"
cross_encoder = CrossEncoder(model_name, num_labels=1, device=device)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
optimizer = torch.optim.AdamW(cross_encoder.model.parameters(), lr=2e-5)

margin = 1   # MarginRankingLoss 값
loss_fn = torch.nn.MarginRankingLoss(margin=margin)


# =========================================
# 6. 학습 루프 (직접 구현)
# =========================================

epochs = 2

for epoch in range(epochs):
    total_loss = 0
    cross_encoder.model.train()

    for q, pos, neg in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):

        # ========= Positive input =========
        pos_inputs = tokenizer(
            list(q),
            list(pos),
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        pos_output = cross_encoder.model(**pos_inputs)
        pos_scores = pos_output.logits.squeeze()   # (batch,)

        # ========= Negative input =========
        neg_inputs = tokenizer(
            list(q),
            list(neg),
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        neg_output = cross_encoder.model(**neg_inputs)
        neg_scores = neg_output.logits.squeeze()   # (batch,)

        # ========= Margin Ranking Loss =========
        # pos > neg 되어야 하므로 target = 1
        target = torch.ones(pos_scores.size(0)).to(device)

        loss = loss_fn(pos_scores, neg_scores, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {total_loss / len(loader):.4f}")

print("=== Training Finished ===")

# =========================================
# 7. 모델 저장
# =========================================
cross_encoder.save("./htp_cross_encoder_marginranking")
print("모델 저장 완료.")


Using device: cuda
Positive pairs: 573


100%|██████████| 573/573 [00:05<00:00, 101.14it/s]


Triples: 1146


Epoch 1/2: 100%|██████████| 287/287 [00:26<00:00, 10.66it/s]


[Epoch 1] Loss: 0.1780


Epoch 2/2: 100%|██████████| 287/287 [00:26<00:00, 10.78it/s]


[Epoch 2] Loss: 0.0525
=== Training Finished ===
모델 저장 완료.


In [18]:
# =========================================
# 8. 평가 함수
# =========================================
import numpy as np

def calculate_similarity(model, tokenizer, query, candidates, max_length=256):
    model.eval()
    raw_scores = []

    with torch.no_grad():
        for cand in candidates:
            inputs = tokenizer(
                query,
                cand,
                padding='max_length',
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            ).to(device)

            outputs = model(**inputs)
            logit = outputs.logits.squeeze().item()   # raw score
            raw_scores.append(logit)

    # 🔥 Softmax 정규화
    scores_tensor = torch.tensor(raw_scores)
    softmax_scores = torch.softmax(scores_tensor, dim=0).tolist()

    # 🔥 candidate + softmax score 묶기
    results = []
    for cand, prob in zip(candidates, softmax_scores):
        results.append((cand, prob))

    # 확률 높은 순으로 정렬
    results = sorted(results, key=lambda x: x[1], reverse=True)
    return results


기존 pairwise 방식은 pos_score를 계속 올리고 neg_score를 계속 내린다
그래서 logits가 너무 발산해서 sigmoid → 0.999 vs 0.000 으로 극단값 발생
MarginRankingLoss는 pos_score ≥ neg_score + margin 까지만 학습하고 그 이상은 fine-tuning을 멈춤
즉, 과도하게 score를 벌리지 않음 → 더 자연스러운 점수 분포 유지

In [19]:
# =========================================
# 9. 검증 데이터 평가
# =========================================
valid_dataset = load_dataset("HJUNN/Art_Therapy_caption_valid_dataset")["validation"]
valid_df = valid_dataset.to_pandas()

test_queries = [
    valid_df["query"][0],
    valid_df["query"][55],
    valid_df["query"][85]
]

test_candidates = [
    valid_df["doc"][0], valid_df["doc"][7], valid_df["doc"][20],
    valid_df["doc"][55], valid_df["doc"][60], valid_df["doc"][65],
    valid_df["doc"][85], valid_df["doc"][90], valid_df["doc"][95],
]

print("\n=== 학습 후 Reranker 점수 ===")
for q in test_queries:
    print(f"\n[Query] {q}")
    scores = calculate_similarity(model, tokenizer, q, test_candidates)
    for i, (doc, s) in enumerate(scores, start=1):
        print(f"{i}위 | 점수 {s:.4f} | 문서: {doc[:60]}...")
    print("-" * 70)

model.save_pretrained("./htp_bge_cross_encoder")
tokenizer.save_pretrained("./htp_bge_cross_encoder")
print("저장 완료: ./htp_bge_cross_encoder")



=== 학습 후 Reranker 점수 ===

[Query] 그림의 집은 넓고 크게 그려져 있으며, 중앙에 자리 잡고 있습니다.
1위 | 점수 0.9331 | 문서: 1.<집 그림의 분석결과 정리>


 집 크기 
상태: 크게 그리는 경우 
가능한 분석: 피험자가 상상력이 ...
2위 | 점수 0.0184 | 문서: 3. 항목: 집의 형태 
상태: 양옥집 
가능한 분석: 양옥집을 그리는 피험자의 경우 타인의 말에 잘 공감해...
3위 | 점수 0.0155 | 문서: 7. 항목: 창문 크기 
상태: 크게 
가능한 분석: 피험자가 호기심이 많을수록, 야심이 있고 목표지향적일수...
4위 | 점수 0.0110 | 문서: 7. 가지모양 
 - (가지모양 이미지) 피험자가 창의적일수록, 공감을 잘할수록 위로 뾰족한 가지를 그리는 ...
5위 | 점수 0.0070 | 문서: 5. 동물유무 
- 있다: 피험자가 자신감이 클수록 동물을 그리는 경향이 있다.
 
뿌리위치 
- 지면아래:...
6위 | 점수 0.0064 | 문서: 8. 줄기에서 뻗은 새끼가지 
있다: 피험자가 야심 있고 목표지향적일수록 줄기에서 뻗은 새끼 가지를 그리는 ...
7위 | 점수 0.0037 | 문서: 11. 다리그림
- 짧다: 피험자가 현실적일수록 다리를 짧게 그리는 경향이 있다.


다리모양
- 나란히 붙...
8위 | 점수 0.0031 | 문서: 9. 두상비율
- 적당: 피험자가 성취동기가 높을수록 두상비율을 균형 있게 그리는 경향이 있다.
- 크다: ...
9위 | 점수 0.0017 | 문서: 7. 목그림
- 생략: 피험자가 우울할수록, 반사회적 성향이 강할수록, 또는 인내심이 강할수록 목을 그리지 ...
----------------------------------------------------------------------

[Query] 나무 뿌리는 땅 속 깊이 자리 잡고 있으며, 여백에는 잔디가 고르게 깔려 있습니다. 동물이 나무 옆에 서 있는 모습도 보입니다.
1위 | 점수 0.